In [1]:
from vespa.package import HNSW, ApplicationPackage, Component, Field, Parameter

app_name = "snowflake"

app_package = ApplicationPackage(
    name=app_name,
    components=[
        Component(
            id="snow",
            type="hugging-face-embedder",
            parameters=[
                Parameter(
                    "transformer-model",
                    {
                        "url": "https://huggingface.co/Snowflake/snowflake-arctic-embed-l/resolve/main/onnx/model_int8.onnx"
                    },
                ),
                Parameter(
                    "tokenizer-model",
                    {
                        "url": "https://huggingface.co/Snowflake/snowflake-arctic-embed-l/raw/main/tokenizer.json"
                    },
                ),
                Parameter(
                    "normalize",
                    {},  # Define as a simple string, not a dictionary
                    "true",  # Define as a simple string, not a dictionary
                ),
                Parameter(
                    "pooling-strategy",
                    {},
                    "cls",  # Define as a simple string, not a dictionary
                ),
            ],
        )
    ],
)

In [2]:
app_package.schema.add_fields(
    Field(name="id", type="int", indexing=["attribute", "summary"]),
    Field(
        name="doc", type="string", indexing=["index", "summary"], index="enable-bm25"
    ),
    Field(
        name="doc_embeddings",
        type="tensor<float>(x[384])",
        indexing=["input doc", "embed", "index", "attribute"],
        ann=HNSW(distance_metric="prenormalized-angular"),
        is_document_field=False,
    ),
)

In [3]:
from vespa.package import (
    DocumentSummary,
    FieldSet,
    FirstPhaseRanking,
    RankProfile,
    SecondPhaseRanking,
    Summary,
)

app_package.schema.add_rank_profile(
    RankProfile(
        name="semantic",
        inputs=[("query(q)", "tensor<float>(x[384])")],
        inherits="default",
        first_phase="closeness(field, doc_embeddings)",
        match_features=["closeness(field, doc_embeddings)"],
    )
)

app_package.schema.add_rank_profile(RankProfile(name="bm25", first_phase="bm25(doc)"))

In [4]:
app_package.schema.add_rank_profile(
    RankProfile(
        name="hybrid",
        inherits="semantic",
        functions=[],
        first_phase=FirstPhaseRanking(expression="closeness(field, doc_embeddings)"),
        # Notice that we use log10 here, as the bm25 values with the natural logarithm tends to dominate the closeness values for these documents.
        second_phase=SecondPhaseRanking(expression="firstPhase + log10( bm25(doc))"),
        match_features=[
            "firstPhase",
            "bm25(doc)",
        ],
    )
)

In [5]:
app_package.schema.add_field_set(FieldSet(name="default", fields=["doc"]))

In [6]:
app_package.schema.add_document_summary(
    DocumentSummary(
        name="minimal",
        summary_fields=[Summary("id", "int"), Summary("doc", "string")],
    )
)

Create some sample documents that will help us see where the different ranking strategies have their strengths and weaknesses.

> These documents were created by ChatGPT.

Looking through the documents, we can see that a ranking of the documents in the order they are presented seem quite reasonable.


In [31]:
# Query that the user is searching for
query = "How does Vespa handle real-time indexing and search?"

# List of documents simulating content in the search engine
documents = [
    "Vespa excels in real-time data indexing and its ability to search large datasets quickly.",
    "Instant data availability and maintaining query performance while simultaneously indexing are key features of the Vespa search engine.",
    "With our search solution, real-time updates are seamlessly integrated into the search index, enhancing responsiveness.",
    "While not as robust as Vespa, our vector database strives to meet your search needs, despite certain, shall we say, 'flexible' features.",
    "Search engines like ours utilize complex algorithms to handle immediate data querying and indexing.",
    "Modern search platforms emphasize quick data retrieval from continuously updated indexes.",
    "Discover the history and cultural impact of the classic Italian Vespa scooter brand.",
    "Tips for maintaining your Vespa to ensure optimal performance and longevity of your scooter.",
    "Review of different scooter brands including Vespa, highlighting how they handle features like speed, cost, and aesthetics, and how consumers search for the best options.",
    "Vespa scooter safety regulations and best practices for urban commuting.",
]

## Dumping the application package to files

This is a good practice to inspect and understand the structure of the application package and schema files, generated by pyvespa.


In [32]:
app_package.to_files("snowflake")

In [33]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy(app_package)

Waiting for configuration server, 0/60 seconds...
Waiting for configuration server, 5/60 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 10/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Application is up!
Finished deployment.


In [34]:
feed_docs = [
    {
        "id": str(i),
        "fields": {
            "doc": doc,
        },
    }
    for i, doc in enumerate(documents)
]

In [35]:
from vespa.io import VespaResponse


def callback(response: VespaResponse, id: str):
    if not response.is_successful():
        print(
            f"Failed to feed document {id} with status code {response.status_code}: Reason {response.get_json()}"
        )

In [36]:
app.feed_iterable(feed_docs, schema=app_package.schema.name, callback=callback)

Let us use the [Vespa CLI](https://docs.vespa.ai/en/vespa-cli) `visit` command to retrieve all indexed documents.


In [37]:
!vespa visit

{"id":"id:snowflake:snowflake::2","fields":{"doc":"With our search solution, real-time updates are seamlessly integrated into the search index, enhancing responsiveness."}}
{"id":"id:snowflake:snowflake::4","fields":{"doc":"Search engines like ours utilize complex algorithms to handle immediate data querying and indexing."}}
{"id":"id:snowflake:snowflake::1","fields":{"doc":"Instant data availability and maintaining query performance while simultaneously indexing are key features of the Vespa search engine."}}
{"id":"id:snowflake:snowflake::5","fields":{"doc":"Modern search platforms emphasize quick data retrieval from continuously updated indexes."}}
{"id":"id:snowflake:snowflake::3","fields":{"doc":"While not as robust as Vespa, our vector database strives to meet your search needs, despite certain, shall we say, 'flexible' features."}}
{"id":"id:snowflake:snowflake::6","fields":{"doc":"Discover the history and cultural impact of the classic Italian Vespa scooter brand."}}
{"id":"id:

## Compute a metric

If we assume that the documents are created in decreasing order of relevance, we can compute the Normalized Discounted Cumulative Gain (NDCG) metric for the different ranking strategies.

The NDCG is a measure of ranking quality. It is calculated as the sum of the discounted gain of the relevant documents, divided by the ideal DCG. The ideal DCG is the DCG of the perfect ranking, where the documents are ordered by relevance.

The formula for NDCG is:

$$
NDCG = \frac{DCG}{IDCG}
$$

where:

$$
DCG = \sum_{i=1}^{n} \frac{2^{rel_i} - 1}{\log_2(i + 1)}
$$


In [38]:
import math
from typing import List


def ndcg_at_k(rank_order: List[int], ideal_order: List[int], k: int) -> float:
    """
    Calculate the normalized Discounted Cumulative Gain (nDCG) at position k.

    Parameters:
        rank_order (List[int]): The list of document indices as ranked by the search system.
        ideal_order (List[int]): The list of document indices in the ideal order.
        k (int): The position up to which to calculate nDCG.

    Returns:
        float: The nDCG value at position k.
    """
    dcg = 0.0
    idcg = 0.0

    # Calculate DCG based on the ranked order up to k
    for i in range(min(k, len(rank_order))):
        rank_index = rank_order[i]
        # Find the rank index in the ideal order to assign relevance
        if rank_index in ideal_order:
            relevance = len(ideal_order) - ideal_order.index(rank_index)
        else:
            relevance = 0
        dcg += relevance / math.log2(i + 2)

    # Calculate IDCG based on the ideal order up to k
    for i in range(min(k, len(ideal_order))):
        relevance = len(ideal_order) - i
        idcg += relevance / math.log2(i + 2)

    # Handle the case where IDCG is zero to avoid division by zero
    if idcg == 0:
        return 0.0
    return dcg / idcg


# Example usage
rank_order = [5, 6, 1]  # Example ranked order indices
ideal_result_order = [0, 1, 2, 4, 5, 3, 6, 7, 8, 9]  # Example ideal order indices

# Calculate nDCG@3
result = ndcg_at_k(rank_order, ideal_result_order, 3)
print(f"nDCG@3: {result:.4f}")

nDCG@3: 0.6618


In [49]:
# Define the different rank profiles to evaluate

rank_profiles = {
    "unranked": {
        "yql": f"select * from {app_name} where true",
        "ranking.profile": "unranked",
    },
    "bm25": {
        "yql": f"select * from {app_name} where userQuery()",
        "ranking.profile": "bm25",
    },
    "semantic": {
        "yql": f"select * from {app_name} where {{targetHits:5}}nearestNeighbor(doc_embeddings,q)",
        "ranking.profile": "semantic",
        "input.query(q)": f"embed({query})",
    },
    "hybrid": {
        "yql": f"select * from {app_name} where userQuery() or ({{targetHits:5}}nearestNeighbor(doc_embeddings,q))",
        "ranking.profile": "hybrid",
        "input.query(q)": f"embed({query})",
    },
    "hybrid_filtered": {
        "yql": f'select * from {app_name} where !(doc contains "scooter") and userQuery() or ({{targetHits:5}}nearestNeighbor(doc_embeddings,q))',
        "ranking.profile": "hybrid",
        "input.query(q)": f"embed({query})",
    },
}

# Define some common params that will be used for all queries

common_params = {
    "query": query,
    "hits": 3,
}

In [50]:
import json
from typing import List, Tuple

from vespa.application import Vespa
from vespa.io import VespaQueryResponse


def evaluate_rank_profile(
    app: Vespa, rank_profile: str, params: dict, k: int
) -> Tuple[float, List[str]]:
    """
    Run a query against a Vespa application using a specific rank profile and parameters.
    Evaluate the nDCG@3 of the search results based on the ideal order.

    Parameters:
        app (Vespa): The Vespa application to query.
        rank_profile (str): The name of the rank profile to use.
        params (dict): The common parameters to use in addition to the rank profile specific parameters.
        k (int): The position up to which to calculate nDCG.

    Returns:
        List[str]: The search results
    """
    body_params = {
        **rank_profiles[rank_profile],
        **params,
    }
    response: VespaQueryResponse = app.query(body_params)
    rankings = [int(hit["id"][-1]) for hit in response.hits]
    docs = [hit["fields"]["doc"] for hit in response.hits]
    ndcg = ndcg_at_k(rankings, ideal_order=ideal_result_order, k=3)
    return ndcg, docs

In [52]:
for rank_profile, params in rank_profiles.items():
    ndcg, docs = evaluate_rank_profile(
        app, rank_profile=rank_profile, params=common_params, k=3
    )
    print(f"Rank profile: {rank_profile}, nDCG@3: {ndcg:.2f}")
    print(json.dumps(docs, indent=2))

Rank profile: unranked, nDCG@3: 0.66
[
  "Modern search platforms emphasize quick data retrieval from continuously updated indexes.",
  "Discover the history and cultural impact of the classic Italian Vespa scooter brand.",
  "Instant data availability and maintaining query performance while simultaneously indexing are key features of the Vespa search engine."
]
Rank profile: bm25, nDCG@3: 0.78
[
  "Vespa excels in real-time data indexing and its ability to search large datasets quickly.",
  "Review of different scooter brands including Vespa, highlighting how they handle features like speed, cost, and aesthetics, and how consumers search for the best options.",
  "With our search solution, real-time updates are seamlessly integrated into the search index, enhancing responsiveness."
]
Rank profile: semantic, nDCG@3: 1.00
[
  "Vespa excels in real-time data indexing and its ability to search large datasets quickly.",
  "Instant data availability and maintaining query performance while s

In short, the above query demonstrates how easy it is to combine various ranking strategies,
and also combine with filters.

To learn more about pre-filtering vs post-filtering,
read [Filtering strategies and serving performance](https://blog.vespa.ai/constrained-approximate-nearest-neighbor-search/).
[Semantic search with multi-vector indexing](https://blog.vespa.ai/semantic-search-with-multi-vector-indexing/)
is a great read overall for this domain.


## Next steps

Check out global reranking strategies, and try to introduce a global_phase reranking strategy.


## Cleanup


In [23]:
# vespa_docker.container.stop()
# vespa_docker.container.remove()